In [162]:
import numpy as np
import pandas as pd
import periodictable

from pathlib import Path

In [163]:
coeff_df = pd.read_csv('https://raw.githubusercontent.com/HackTheSolarSystem/MineralMapping/master/challenge_data/weight_proportion_coefficients.csv')
coeff_df

,element,coefficient
0,Mg,0.001186
1,Ni,0.003291
2,Fe,0.003189
3,Ca,0.002151
4,S,0.004667
5,Ti,0.000781
6,Si,0.000982


In [164]:
chemical_formulas = {
    "Kamacite": "Fe9Ni",
    "Taenite": "Fe8Ni2",
    "Millerite": "NiS",
    "Troilite": "FeS",
    "Pentlandite": "FeNiS",
    "Olivine": "Fe2Mg2SiO4",
    "Pyroxene": "FeMgSiO3",
}

In [165]:
weight_proportions = {
    mineral: {
        str(e): m
        for e, m in periodictable.formula(formula).mass_fraction.items()
    }
    for mineral, formula in chemical_formulas.items()
}
weight_proportions["Olivine"] = {
    'Ca': 0.0002,
    'Fe': 0.0742,
    'Mg': 0.298,
    'Mn': 0.0011,
    'Ni': 0.0029,
    'Si': 0.1908,
}
weight_proportions_df = pd.DataFrame.from_records(weight_proportions).fillna(0)

In [166]:
weight_proportion_elements = set(weight_proportions_df.index)
coeff_elements = set(coeff_df.element.unique())

# Remove any elements that don't appear in the coefficients
missing_elements = list(weight_proportion_elements - coeff_elements)
weight_proportions_df.drop(missing_elements, inplace=True)

# Add in empty rows for elements that appear in the coefficients but
# are missing from the weight proportions
extra_elements = list(coeff_elements - weight_proportion_elements)
for e in extra_elements:
    weight_proportions_df.loc[e] = [0] * weight_proportions_df.shape[1]
weight_proportions_df

,Kamacite,Millerite,Olivine,Pentlandite,Pyroxene,Taenite,Troilite
Ca,0.000000,0.000000,0.0002,0.000000,0.000000,0.000000,0.000000
Fe,0.895433,0.000000,0.0742,0.380926,0.357445,0.791922,0.635252
Mg,0.000000,0.000000,0.2980,0.000000,0.155568,0.000000,0.000000
Ni,0.104567,0.646699,0.0029,0.400355,0.000000,0.208078,0.000000
S,0.000000,0.353301,0.0000,0.218719,0.000000,0.000000,0.364748
Si,0.000000,0.000000,0.1908,0.000000,0.179766,0.000000,0.000000
Ti,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [167]:
weight_proportions_df.sort_index(inplace=True)
coeff_df.sort_values('element', inplace=True)

coeff_vec = np.reciprocal(np.array(coeff_df.coefficient.values))
theor_intensities = np.diag(coeff_vec).dot(weight_proportions_df).T
theor_intensities

array([[0.00000000e+00, 2.80754999e+02, 0.00000000e+00, 3.17720337e+01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.96495219e+02,
        7.57064867e+01, 0.00000000e+00, 0.00000000e+00],
       [9.29880613e-02, 2.32647507e+01, 2.51251746e+02, 8.81145339e-01,
        0.00000000e+00, 1.94208412e+02, 0.00000000e+00],
       [0.00000000e+00, 1.19435863e+02, 0.00000000e+00, 1.21645144e+02,
        4.68679417e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.12073794e+02, 1.31163722e+02, 0.00000000e+00,
        0.00000000e+00, 1.82977255e+02, 0.00000000e+00],
       [0.00000000e+00, 2.48299961e+02, 0.00000000e+00, 6.32232308e+01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.99177608e+02, 0.00000000e+00, 0.00000000e+00,
        7.81594767e+01, 0.00000000e+00, 0.00000000e+00]])

In [168]:
from sklearn.neighbors import KNeighborsClassifier

In [169]:
classes = range(theor_intensities.shape[0])

# Fit a nearest neightbors classifier with the theoretical intensities as the centers
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(theor_intensities, classes)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [196]:
from skimage.io import imread, imshow
from pathlib import Path

image_path = Path("obj2")
obj1_intensities = [
    {
        'name': s.name.split('_')[2].split('.')[0],
        'image': imread(s)
    }
    for s in image_path.glob('*32bt*')
    if s.name.split('_')[2].split('.')[0] in weight_proportions_df.index
]
obj1_intensities

[{'name': 'Ca', 'image': array([[12, 12, 11, ...,  5,  4, 12],
         [ 4,  4, 15, ...,  4,  8,  0],
         [ 6,  6, 18, ...,  5,  3,  7],
         ...,
         [ 0,  3,  0, ..., 10,  7,  4],
         [ 0,  0,  0, ...,  3,  7,  2],
         [ 2,  0,  2, ...,  7,  3,  3]], dtype=int32)},
 {'name': 'Fe', 'image': array([[49, 44, 45, ..., 19, 21, 17],
         [38, 42, 48, ..., 11, 17, 23],
         [52, 38, 37, ..., 20, 17, 20],
         ...,
         [48, 33, 30, ..., 41, 41, 41],
         [47, 46, 34, ..., 42, 54, 43],
         [56, 32, 41, ..., 56, 51, 40]], dtype=int32)},
 {'name': 'Mg', 'image': array([[ 66,  76,  90, ..., 167, 171, 150],
         [ 49,  62,  90, ..., 159, 174, 148],
         [ 70,  72,  64, ..., 186, 159, 181],
         ...,
         [239, 201, 196, ..., 114, 133, 186],
         [216, 162, 158, ...,  90, 127, 186],
         [188, 149, 161, ...,  79, 121, 138]], dtype=int32)},
 {'name': 'Ni', 'image': array([[2, 0, 2, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 2

In [197]:
dfs = []

pixels = []
for element in obj1_intensities:
    #image = imread(s2)
    pixels.append(element['image'])

#df = pd.DataFrame(np.dstack(pixels)[0], columns=[i['name'] for i in obj1_intensities])
#df

points = np.dstack(pixels)
points

array([[[ 12,  49,  66, ...,   0, 330,   3],
        [ 12,  44,  76, ...,   0, 226,   0],
        [ 11,  45,  90, ...,   0, 221,   0],
        ...,
        [  5,  19, 167, ...,   0, 263,   0],
        [  4,  21, 171, ...,   0, 271,   3],
        [ 12,  17, 150, ...,   0, 289,   0]],

       [[  4,  38,  49, ...,   0, 344,   0],
        [  4,  42,  62, ...,   0, 215,   4],
        [ 15,  48,  90, ...,   0, 210,   4],
        ...,
        [  4,  11, 159, ...,   0, 297,   4],
        [  8,  17, 174, ...,   0, 288,   0],
        [  0,  23, 148, ...,   0, 286,   3]],

       [[  6,  52,  70, ...,   0, 307,   2],
        [  6,  38,  72, ...,   0, 231,   3],
        [ 18,  37,  64, ...,   0, 183,   3],
        ...,
        [  5,  20, 186, ...,   0, 275,   2],
        [  3,  17, 159, ...,   0, 281,   4],
        [  7,  20, 181, ...,   0, 285,   2]],

       ...,

       [[  0,  48, 239, ...,   0, 194,   0],
        [  3,  33, 201, ...,   0, 164,   0],
        [  0,  30, 196, ...,   0, 191,   2

In [198]:
color_map = [
    [255, 0, 0],
    [255, 165, 0],
    [255, 255, 0],
    [160, 82, 45],
    [0, 0, 255],
    [128, 0, 128],
    [255, 128, 0],
]

img_pts = []
for row in points:
    row_pts = []
    y_hat = clf.predict(row)
    for val in y_hat:
        rgb = color_map[val]
        row_pts.append(rgb)
#    row_pts.append(color_map[int(clf.predict(row))])
    img_pts.append(row_pts)
np.array(img_pts)

array([[[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [255, 255,   0],
        [255, 255,   0],
        [255, 255,   0]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [255, 255,   0],
        [255, 255,   0],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [255, 255,   0],
        [255, 255,   0],
        [255, 255,   0]],

       ...,

       [[255, 255,   0],
        [255, 255,   0],
        [255, 255,   0],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [255, 255,   0]],

       [[255, 255,   0],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [255, 255,   0]],

       [[255, 255,   0],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]]

In [199]:
from PIL import Image
img = Image.fromarray(np.array(img_pts).astype(np.uint8))

In [200]:
img.show()

In [201]:
img.save("/Users/peter/sandbox/MineralMapping/backend/obj2_NN.png")